### slice 객체

In [1]:
interval2 = slice(None, None, 2)

In [2]:
_0_to_100_range = range(101)
print(_0_to_100_range[interval2])

range(0, 101, 2)


In [3]:
_0_to_100_list = list(range(101))
print(_0_to_100_list[interval2])

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100]


### 자체 시퀀스를 만들 때 고려할 점

- `_getitem__`과 `__len__` 구현
- 슬라이싱의 결과는 항상 같은 타입의 인스턴스일 것
- 슬라이싱에 의해 제공된 범위의 마지막 인덱스는 제외할 것

### 컨텍스트 관리자

- 리소스 관리에 좋음
- 관심사를 분리하고 독립적으로 유지되어야하는 코드를 분리하는 좋은 방법
- 구현하려면 `__enter__`와 `__exit__`를 구현해야함

In [4]:
def stop_db():
    print("systemctl stop postgresql.service")
    
def start_db():
    print("systemctl start postgresql.service")
    
# define context manager
class DBHandler:
    def __enter__(self):
        stop_db()
        return self
    
    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_db()
        
def backup_db():
    print("pg_dump database")
    
def backup_db_error():
    try:
        raise BlockingIOError("Error: IO is blocked")
    except BlockingIOError as e:
        print(e)
    
def main_normal():
    with DBHandler():
        backup_db()
        
def main_error():
    with DBHandler():
        backup_db_error()

In [5]:
main_normal()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


In [6]:
main_error()

systemctl stop postgresql.service
Error: IO is blocked
systemctl start postgresql.service


- 위 예시에서 에러가 발생해도 db는 다시 시작됨

### contextlib을 사용한 contextmanager 구현

In [7]:
import contextlib

@contextlib.contextmanager
def db_handler():
    stop_db()
    yield
    start_db()

- `@contextlib.contextmanager`: attach된 함수를 context manager로 바꿔줌


- `yield` 위로는 `__enter__` 역할
- `yield` 에서는 반환할 게 있으면 반환, 없으면 지금처럼
- `yield` 아래로는 `__exit__` 역할

In [8]:
with db_handler():
    backup_db()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


In [9]:
with db_handler():
    backup_db_error()

systemctl stop postgresql.service
Error: IO is blocked
systemctl start postgresql.service


-  `contextlib.ContextDecorator`를 상속해서 decorator로 쓰이는 클래스를 정의할 수 있지만, `__enter__`의 결과 값을 반환할 수 없어서 일반적이지 못함

### name mangling

- 파이썬의 class 내에서 변수 앞에 `__`를 붙이는 경우, `_{class-name}__{variable-name}`의 속성을 만드는 것, private과 유사하게 동작한다.

In [10]:
class Connector:
    def __init__(self, source):
        self.sourcce = source
        self.__timeout = 60
        
conn = Connector('postgresql://localhost')

In [19]:
conn = Connector('postgresql://localhost')
print(conn._Connector__timeout)
print(conn.__timeout)

60


AttributeError: 'Connector' object has no attribute '__timeout'

In [11]:
conn._Connector__timeout

60

In [12]:
conn.__timeout

AttributeError: 'Connector' object has no attribute '__timeout'

In [13]:
conn.__dict__

{'sourcce': 'postgresql://localhost', '_Connector__timeout': 60}

### property

In [17]:
import re as regex

EMAIL_FORMAT = regex.compile(r"[^@]+@[^@]+[^@]+")

class User:
    def __init__(self, username):
        self.username = username
        self._email = None
        
    @property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, new_email):
        if not self.is_vaild_email(new_email):
            raise ValueError(f'invalid email format')
        self._email = new_email
    
    @staticmethod
    def is_vaild_email(potentially_vaild_email: str) -> bool:
        return regex.match(EMAIL_FORMAT, potentially_vaild_email) is not None
    

In [19]:
u1 = User('john')
u1.email = 'god_j@'

ValueError: invalid email format

In [20]:
u2 = User('jay')
u2.email = 'cooljay@gamil.com'
u2.email

'cooljay@gamil.com'

- 객체의 모든 속성에 `@property`를 할 필요 없고, 속성 값을 가져오거나 수정할 때 특별한 로직이 필요한 경우에만 사용 권장

> 명령-쿼리 분리 원칙 (command and query seperation)
- 객체의 메서드가 무언가의 상태를 변경하는 커맨드이거나 무언가의 값을 반환하는 쿼리이거나 둘 중에 하나만 수행해야지 둘 다 동시에 수행하면 안된다는 원칙
- `@property`는 *query*, `@{var_name}.setter`는 *command*

### 이터러블 객체

- `for e in elements:`를 수행할 수 있는 객체
    - Check1: `__next__` 또는 `__iter__` 중 하나를 포함
    - Check2: `__len__`과 `__getitem__`을 모두 포함


- `__iter__`: for iterable
- `__next__`: for iterater


- (용어) fallback mechanism: 만일을 대비해 준비한 절차

In [35]:
from datetime import date, timedelta

class DateRangeIterable:
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._present_day = start_date
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._present_day >= self.end_date:
            raise StopIteration
        today = self._present_day
        self._present_day += timedelta(days=1)
        return today

In [36]:
dri =  DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))

In [37]:
for day in dri:
    print(day)

2019-01-01
2019-01-02
2019-01-03
2019-01-04


In [47]:
max(dri)

ValueError: max() arg is an empty sequence

- data를 모두 iterate하고 나면, reload할 수 없는 단점 존재

In [39]:
from datetime import date, timedelta

class DateRangeContainerIterable:
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._present_day = start_date
        
    def __iter__(self):
        current_day = self.start_date
        while current_day < self.end_date:
            yield current_day
            current_day += timedelta(days=1)

In [40]:
drci = DateRangeContainerIterable(date(2019, 1, 1), date(2019, 1, 5))

In [41]:
for day in drci:
    print(day)

2019-01-01
2019-01-02
2019-01-03
2019-01-04


In [46]:
max(drci)

datetime.date(2019, 1, 4)

- `__iter__`에서 제너레이터를 구현해서 이전의 문제 해결
- 이를 container iterable이라고 함

### 시퀀스 만들기

- 즉 인덱싱과 슬라이싱이 가능한 객체 만들기
- 이터러블 객체보다 메모리는 더 잡아먹지만, 검색 속도가 빠름

In [48]:
class DateRangeSequence:
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._range = self._create_range()
    
    def _create_range(self):
        days = []
        current_day = self.start_date
        while current_day < self.end_date:
            days.append(current_day)
            current_day += timedelta(days=1)
        return days
 
    def __getitem__(self, day_no):
        return self._range[day_no]
    
    def __len__(self):
        return len(self._range)

In [49]:
drs = DateRangeSequence(date(2019, 1, 1), date(2019, 1, 5))

In [50]:
for day in drs:
    print(day)

2019-01-01
2019-01-02
2019-01-03
2019-01-04


In [51]:
drs[0]

datetime.date(2019, 1, 1)

- 다른 iterable inatance들 (dri, drci)는 검색 불가

### 컨테이너 객체

- membership을 확인할 수 있는 `in`연산이 가능한 객체
- 내부에 `__contains__`구현 필요

In [53]:
# matrix를 이동하는 문제를 풀 때, 이동 범위를 벗어나는지 아닌지를 pythonic하게 확인할 수 있음

class Boundaries:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        
    def __contains__(self, coord):
        x, y = coord
        return 0 <= x < self.width and 0 <= y < self.height
    
class Grid:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.limits = Boundaries(width, height)
        
    def __contains__(self, coord):
        return coord in self.limits

In [54]:
g = Grid(4, 5)

In [55]:
(0, 3) in g

True

In [56]:
(5, 5) in g

False

### 객체의 동적 속성

- `__getattr__`: 속성을 **조회**할 때, 객체가 갖고 있지 않는 속성일 경우 실행되는 로직

In [84]:
class DynaminAttributes:
    def __init__(self, attribute):
        self.attribute = attribute
        
    def __getattr__(self, attr):
        if attr.startswith('fallback_'):
            name = attr.replace('fallback_', '')
            return f'[fallback resolved] {name}'
        raise AttributeError(f'{self.__class__.___name__} 에는 {attr} 속성이 없음')

In [74]:
da = DynaminAttributes('old_value')

In [75]:
da.attribute

'old_value'

In [76]:
da.fallback_test

'[fallback resolved] test'

In [77]:
da.__dict__

{'attribute': 'old_value'}

In [78]:
da.fallback_test = 'big_value'

In [79]:
da.__dict__

{'attribute': 'old_value', 'fallback_test': 'big_value'}

In [80]:
da.__dict__['fallback_new'] = 'new_value'

In [81]:
da.__dict__

{'attribute': 'old_value',
 'fallback_test': 'big_value',
 'fallback_new': 'new_value'}

In [82]:
getattr(da, 'attribute', 'default')

'old_value'

In [83]:
getattr(da, 'unknown', 'default')

'default'

### callable instance

- instance를 함수처럼 호출할 수 있게 해줌
- `__call__`구현해야함
- 또는 `@callable`로도 대체 가능
- 자세한 내용은 나중에

### python tips

> 함수의 기본 파라미터 값을 dict, list와 같은 객체 값으로 주면, 이들은 처음에만 생성되고 그 뒤로는 계속 남아있는다. 따라서 이들을 수정하는 로직이 함수안에 있을 경우, 다음 호출 시의 결과를 에측하고 어렵고, 에러가 발생할 수 있다.

> `list, str, dict` 와 같은 built-in class의 기능을 확장하려면, `collections` 라이브러이의 `UserList, UserString, UserDict`를 상속해야, 오버라이드 하지않은 나머지 함수들도 제대로 작동할 수 있다.

In [105]:
class BadList(list):
    def __getitem__(self, index):
        value = super().__getitem__(index)
        if index % 2 == 0:
            prefix = 'even'
        else:
            prefix = 'odd'
        return f'[{prefix}] {value}'

In [106]:
bl = BadList([0, 1, 2, 3, 4, 5])

In [107]:
bl[0]

'[even] 0'

In [108]:
bl[1]

'[odd] 1'

In [109]:
" ".join(bl)

TypeError: sequence item 0: expected str instance, int found

In [110]:
bl

[0, 1, 2, 3, 4, 5]

In [111]:
from collections import UserList

class GoodList(UserList):
    def __getitem__(self, index):
        value = super().__getitem__(index)
        if index % 2 == 0:
            prefix = 'even'
        else:
            prefix = 'odd'
        return f'[{prefix}] {value}'

In [112]:
gl = GoodList([0, 1, 2, 3, 4, 5])

In [113]:
gl

[0, 1, 2, 3, 4, 5]

In [114]:
gl[0]

'[even] 0'

In [115]:
" ".join(gl)

'[even] 0 [odd] 1 [even] 2 [odd] 3 [even] 4 [odd] 5'